In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Price of House/train.csv')

In [ ]:
target = train_df['SalePrice']
id = train_df['Id']

In [ ]:
target_norm = (target - target.mean()) / target.std()

In [ ]:
train_df = train_df.drop(['Id', 'SalePrice', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt'], axis=1)

In [ ]:
for column in train_df.columns:
  if train_df[column].dtype == 'O':
    train_df = train_df.drop(columns = column, axis = 1)

In [ ]:
train_df.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,706,0,150,856,...,548,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,978,0,284,1262,...,460,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,486,0,434,920,...,608,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,216,0,540,756,...,642,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,655,0,490,1145,...,836,192,84,0,0,0,0,0,12,2008


In [ ]:
norm_train_df = (train_df - train_df.mean()) / train_df.std()

In [ ]:
import torch
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(33, 18)
    self.act1 = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(18, 9)
    self.act2 = torch.nn.ReLU()
    self.fc3 = torch.nn.Linear(9, 5)
    self.act3 = torch.nn.ReLU()
    self.fc4 = torch.nn.Linear(5, 1)
  def forward(self, x):
    x = self.fc1(x)
    x = self.act1(x)
    x = self.fc2(x)
    x = self.act2(x)
    x = self.fc3(x)
    x = self.act3(x)
    x = self.fc4(x)
    return x

In [ ]:
net = Net()
optim = torch.optim.Adam(params=net.parameters(), lr=0.01)

In [ ]:
from torch.utils.data import DataLoader
import tensorflow as tf

In [ ]:
data = []

In [ ]:
for i in range(len(train_df)):
  data.append([torch.tensor(norm_train_df.iloc[i], dtype=torch.float64),
               torch.tensor([target_norm[i]], dtype=torch.float64)])

In [ ]:
loader_df = DataLoader(dataset=data, batch_size=20)

In [ ]:
main_loss = torch.nn.MSELoss()

In [ ]:
for epoch in range(150):
  net.train()
  losses = 0
  correct = 0
  for i, (data, target) in enumerate(loader_df):
    optim.zero_grad()
    y_pred = net(data.float())
    # print(y_pred.type(torch.FloatTensor))
    # print(target.type(torch.FloatTensor))
    loss = main_loss(y_pred.type(torch.FloatTensor), target.type(torch.FloatTensor))
    # print(loss)
    loss.backward()
    optim.step()
    losses += loss
    correct += (y_pred == target).float().sum()
  print('Epoch: ', epoch, '     loss: ', loss / len(loader_df), '     accuracy: ', 100 * correct / len(data))

Epoch:  0      loss:  tensor(0.0025, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  1      loss:  tensor(0.0014, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  2      loss:  tensor(0.0009, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  3      loss:  tensor(0.0010, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  4      loss:  tensor(0.0011, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  5      loss:  tensor(0.0010, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  6      loss:  tensor(0.0014, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  7      loss:  tensor(0.0011, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  8      loss:  tensor(0.0011, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  9      loss:  tensor(0.0011, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  10      loss:  tensor(0.0011, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)
Epoch:  11      loss:  tensor(0

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Price of House/test.csv')

In [ ]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
price = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Price of House/sample_submission.csv')

In [ ]:
price.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [ ]:
test = test.drop(['Id', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt'], axis = 1)

In [ ]:
for column in test.columns:
  if test[column].dtype == 'O':
    test = test.drop(columns = column, axis = 1)

In [ ]:
test.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20,11622,5,6,1961,1961,468.0,144.0,270.0,882.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,20,14267,6,6,1958,1958,923.0,0.0,406.0,1329.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,60,13830,5,5,1997,1998,791.0,0.0,137.0,928.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,60,9978,6,6,1998,1998,602.0,0.0,324.0,926.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,120,5005,8,5,1992,1992,263.0,0.0,1017.0,1280.0,...,506.0,0,82,0,0,144,0,0,1,2010


In [ ]:
test_data = []

In [ ]:
price = price.drop(['Id'], axis = 1)

In [ ]:
norm_price = (price - price.mean()) / price.std()

In [ ]:
norm_price

,SalePrice
0,-0.599751
1,0.519089
2,0.266357
3,0.008086
4,-1.722564
...,...
1454,-0.732684
1455,-0.871466
1456,2.423887
1457,0.347515


In [ ]:
target_test = norm_price['SalePrice']

In [ ]:
norm_test = (test - test.mean()) / test.std()

In [ ]:
norm_test

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,-0.874411,0.363804,-0.750844,0.400629,-0.340828,-1.072517,0.063251,0.516994,-0.650173,-0.370554,...,1.185131,0.366553,-0.701387,-0.360615,-0.088796,1.818336,-0.057207,-0.092212,-0.038268,1.713318
1,-0.874411,0.897553,-0.054858,0.400629,-0.439544,-1.214492,1.062663,-0.297698,-0.339145,0.638706,...,-0.740704,2.347063,-0.178765,-0.360615,-0.088796,-0.301440,-0.057207,19.723675,-0.038268,1.713318
2,0.061330,0.809368,-0.750844,-0.497247,0.843770,0.678509,0.772723,-0.297698,-0.954339,-0.266693,...,0.042530,0.930176,-0.207799,-0.360615,-0.088796,-0.301440,-0.057207,-0.092212,-1.140224,1.713318
3,0.061330,0.032053,-0.054858,0.400629,0.876675,0.678509,0.357583,-0.297698,-0.526677,-0.271209,...,-0.012757,2.088735,-0.178765,-0.360615,-0.088796,-0.301440,-0.057207,-0.092212,-0.038268,1.713318
4,1.464941,-0.971475,1.337113,-0.497247,0.679242,0.394559,-0.387033,-0.297698,1.058191,0.528071,...,0.153105,-0.729382,0.489031,-0.360615,-0.088796,2.242291,-0.057207,-0.092212,-1.874861,1.713318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2.400682,-1.590785,-1.446829,1.298505,-0.044678,-0.646592,-0.964715,-0.297698,-0.018970,-1.129193,...,-2.178170,-0.729382,-0.701387,-0.360615,-0.088796,-0.301440,-0.057207,-0.092212,-0.038268,-1.359492
1455,2.400682,-1.599260,-1.446829,-0.497247,-0.044678,-0.646592,-0.411194,-0.297698,-0.595286,-1.129193,...,-0.860493,-0.729382,-0.352972,-0.360615,-0.088796,-0.301440,-0.057207,-0.092212,-0.772905,-1.359492
1456,-0.874411,2.054445,-0.750844,1.298505,-0.373733,0.583859,1.723812,-0.297698,-1.267654,0.401631,...,0.475613,2.981139,-0.701387,-0.360615,-0.088796,-0.301440,-0.057207,-0.092212,1.063688,-1.359492
1457,0.646168,0.125484,-0.750844,-0.497247,0.679242,0.394559,-0.224491,-0.297698,0.047352,-0.302819,...,-2.178170,-0.103134,-0.236834,-0.360615,-0.088796,-0.301440,-0.057207,1.017478,0.329051,-1.359492


In [ ]:
target_test

0      -0.599751
1       0.519089
2       0.266357
3       0.008086
4      -1.722564
          ...   
1454   -0.732684
1455   -0.871466
1456    2.423887
1457    0.347515
1458    0.518089
Name: SalePrice, Length: 1459, dtype: float64

In [ ]:
for i in range(len(test)):
  test_data.append([torch.tensor(norm_test.iloc[i], dtype=torch.float64),
               torch.tensor([target_test[i]], dtype=torch.float64)])

In [ ]:
test_loader = DataLoader(dataset = test_data, batch_size=20)

In [ ]:
for i, (x, y) in enumerate(test_loader):
  y_pred = net(x.float())
  loss = main_loss(y_pred.type(torch.FloatTensor), y.type(torch.FloatTensor))
  losses += loss
  correct += (y_pred == y).float().sum()
print('loss: ', loss / len(loader_df), '     accuracy: ', 100 * correct / len(data))

loss:  tensor(0.0250, grad_fn=<DivBackward0>)      accuracy:  tensor(0.)


In [86]:
test_data[0]

[tensor([-0.8744,  0.3638, -0.7508,  0.4006, -0.3408, -1.0725,  0.0633,  0.5170,
         -0.6502, -0.3706, -0.6543, -0.7750, -0.0805, -1.2152, -0.8187, -0.2583,
         -1.0284, -0.7508, -1.0292, -0.2038, -0.9180, -0.8977, -0.9873,  1.1851,
          0.3666, -0.7014, -0.3606, -0.0888,  1.8183, -0.0572, -0.0922, -0.0383,
          1.7133], dtype=torch.float64),
 tensor([-0.5998], dtype=torch.float64)]